In [315]:
import numpy as np

p_dist = [0.1, 0.2, 0.3, 0.4]
alphabet = ['a', 'b', 'c', 'd']

info_content = lambda p: np.log2(1/p)
entropy = lambda p_dist: np.sum([p*np.log2(1/p) for p in p_dist])


In [327]:
print('Info content of receiving the letter "b" is:',round(info_content(p_dist[1]),4), 'bits')

Info content of receiving the letter "b" is: 2.3219 bits


In [326]:
print('The entropy of the system with probability distribution p_dist is:' ,round(entropy(p_dist),4), 'bits')

The entropy of the system with probability distribution p_dist is: 1.8464 bits


In [93]:
class TreeNode:
    def __init__(self, data, prob, letter=None):
        self.data = data
        self.prob = prob
        self.letter = letter
        self.leftChild = None
        self.rightChild = None
        self.depth = 0
        self.leaf = True

In [312]:
# TODO: build a class with encode and decode methods for the huffman tree
# TODO: try out the ipython UI plugins to make the tree interactive and visualise it

# Implement Huffmans algorithm to find optimal symbol code given an ensemble X w.p P

class HuffmanAlgo:
    
    def __init__(self, alphabet, P):
        self.alphabet = alphabet
        self.P = P
        self.root = None
        self.tree_depth = 0
        self.encoder = {}
        self.treeString = ''

        
    def create_tree(self):
        # create tree from the leaves, starting with the lowest prob combo
        P_alphabet_join = np.concatenate((np.array(self.P).reshape(len(self.P), 1), \
                                          np.array(self.alphabet).reshape(len(self.alphabet), 1)), axis=1)
        P_sorted = np.sort(P_alphabet_join, axis=0)
#         P_sorted = np.sort(self.P)
        P_sorted = [TreeNode(0, prob, letter) for (prob, letter) in P_alphabet_join]

#         P_sorted = [TreeNode(0, prob) for prob in P_sorted]
        leaf_node = P_sorted[0]
        while len(P_sorted) > 1:
            p_0, p_1 = P_sorted[0].prob, P_sorted[1].prob
            p_0_1 = round((float(p_0) + float(p_1)),5)
            p_0_1_node = TreeNode(0, p_0_1)
            p_0_1_node.leftChild = P_sorted[0]
            p_0_1_node.rightChild = P_sorted[1]
            p_0_1_node.rightChild.data = 1
            p_0_1_node.leaf = False
#             latest_treeString = p_0_1_node.print_leaves()
#             print(latest_treeString, 'with root prob', p_0_1)

            P_sorted = P_sorted[2:]
            # need to get sorted list of nodes by prob
            P_sorted_plus_new_node = np.concatenate((P_sorted, [p_0_1_node]), axis=0)
            P_sorted = self._sort_nodes_by_prob(P_sorted_plus_new_node)

        self.root = p_0_1_node
        self.root.data = None
        print(self._print_tree(self.root))
                    
        
        return 'Tree Created Of Depth: {}'.format(self.tree_depth)
    
    def _print_tree(self, node, level=0):
        node.depth = level
        if node.depth > self.tree_depth: self.tree_depth = node.depth

        self.treeString = 'letter: '+str(node.letter)+'--- prob:' +str(node.prob) + '--- data:' +str(node.data) +'--- depth:' + str(node.depth) + \
        '--- leaf: '+str(node.leaf) +'--->\n'
        if node.rightChild:
            self.treeString += self._print_tree(node.rightChild, level+1)
        if node.leftChild:
            self.treeString += self._print_tree(node.leftChild, level+1)
        return self.treeString
    
    def _sort_nodes_by_prob(self, node_arr):
        prob_arr = np.array([node.prob for node in node_arr])
        nodes_plus_probs = np.concatenate((prob_arr.reshape(len(prob_arr),1), node_arr.reshape(len(node_arr), 1)), axis=1 )
        nodes_plus_probs_sorted = nodes_plus_probs[nodes_plus_probs[:, 0].argsort()]
        sorted_nodes = nodes_plus_probs_sorted[:,1]
        return sorted_nodes 

    
    def print_tree(self):      
        latest_treeString = self.root.print_leaves()
        return latest_treeString
    
        
    def _get_path_encodings(self, start_node, path_encoding):
        node = start_node
        if node.leaf is True:
            path_encoding+=str(node.data)
            self.encoder[node.letter] = path_encoding
        else:
            if node is not self.root:
                path_encoding+=str(node.data)
            if node.rightChild:
                self._get_path_encodings(node.rightChild, path_encoding)
            if node.leftChild:
                self._get_path_encodings(node.leftChild, path_encoding)
                
    def create_encoder(self):
        
        self._get_path_encodings(self.root, '')
        print(self.encoder)
        # get all paths to target leaves of tree and append data to encoded_X_dict
        return 'Completed encoder'
    
    def encode_string(self, string_to_encode):
        
        string_to_list = list(string_to_encode)
        enc_string = ''
        for elem in string_to_list:
            try:
                enc_string+=self.encoder[elem]
            except KeyError as err:
                print('Letter not in alphabet. Check encoder is created and please try again.')
                raise
        return enc_string

    
        
            

In [328]:
algoInst = HuffmanAlgo(P=p_dist, alphabet=alphabet)
print(algoInst.create_tree())

letter: None--- prob:1.0--- data:None--- depth:0--- leaf: False--->
letter: None--- prob:0.6--- data:1--- depth:1--- leaf: False--->
letter: None--- prob:0.3--- data:1--- depth:2--- leaf: False--->
letter: b--- prob:0.2--- data:1--- depth:3--- leaf: True--->
letter: a--- prob:0.1--- data:0--- depth:3--- leaf: True--->
letter: c--- prob:0.3--- data:0--- depth:2--- leaf: True--->
letter: d--- prob:0.4--- data:0--- depth:1--- leaf: True--->

Tree Created Of Depth: 3


Pseudocode for encoder:

Input --> tree with nodes and data on each node with leaves corresponding to letter in the alphabet

ALGO -> for all paths in the tree; get letter from the leaf node of the path, concat all data from nodes in path

Output --> function/mapper that takes a letter from the alphabet and converts to binary

In [329]:
algoInst.create_encoder()

{'b': '111', 'a': '110', 'c': '10', 'd': '0'}


'Completed encoder'

In [330]:
# Assume alphabet is a, b, c, d and prob dist = p_dist defined above
test_X = 'aabcddd'

algoInst.encode_string(test_X)

'11011011110000'

In [12]:
import ipywidgets as widgets
from IPython.display import display, clear_output


slider = widgets.IntSlider(
    min=0,
    max=10,
    step=1,
    description='Slider:',
    value=3
)

display(slider)

IntSlider(value=3, description='Slider:', max=10)

In [13]:
slider.value

3

In [14]:
print(dir(widgets))

['Accordion', 'BoundedFloatText', 'BoundedIntText', 'Box', 'Button', 'ButtonStyle', 'CallbackDispatcher', 'Checkbox', 'Color', 'ColorPicker', 'Controller', 'CoreWidget', 'DOMWidget', 'DatePicker', 'Datetime', 'Dropdown', 'FloatLogSlider', 'FloatProgress', 'FloatRangeSlider', 'FloatSlider', 'FloatText', 'HBox', 'HTML', 'HTMLMath', 'Image', 'IntProgress', 'IntRangeSlider', 'IntSlider', 'IntText', 'Label', 'Layout', 'NumberFormat', 'Output', 'Password', 'Play', 'RadioButtons', 'Select', 'SelectMultiple', 'SelectionRangeSlider', 'SelectionSlider', 'SliderStyle', 'Style', 'Tab', 'Text', 'Textarea', 'ToggleButton', 'ToggleButtons', 'ToggleButtonsStyle', 'VBox', 'Valid', 'ValueWidget', 'Widget', '__builtins__', '__cached__', '__doc__', '__file__', '__jupyter_widgets_base_version__', '__jupyter_widgets_controls_version__', '__loader__', '__name__', '__package__', '__path__', '__protocol_version__', '__spec__', '__version__', '_handle_ipython', '_version', 'dlink', 'docutils', 'domwidget', 'fix

In [19]:
# change p_dist and number of variables and create a tree 
# encode and decode any string with a given encoder


create_float_widget = lambda x: widgets.FloatSlider(
    value=1,
    min=0,
    max=1.0,
    step=0.1,
    description='{}:'.format(x),
    disabled=False
)

dimensions_widget = widgets.IntText()
print('Size of alphabet to be coded:')
display(dimensions_widget)

out = widgets.Output()
display(out)
widget_list = []

# Handle changes to the dimension size to prob distribution inputs

def prob_eventhandler(obj):
    global widget_list
    widget_list = [create_float_widget('p{}'.format(i)) for i in range(dimensions_widget.value)]
    with out:
        clear_output()
        print('Probability distribution of alphabet:')
        display(*widget_list)


dimensions_widget.observe(prob_eventhandler, names='value')



Size of alphabet to be coded:


IntText(value=0)

Output()

In [332]:
import string

btn = widgets.Button(description='Create Tree')
display(btn)

out_tree = widgets.Output()
display(out_tree)

def btn_eventhandler(obj):
    p_dist_var = [w.value for w in widget_list]
    alphabet_var = list(string.ascii_lowercase[:len(p_dist_var)])
    global widgetInst
    widgetInst = HuffmanAlgo(P=p_dist_var,alphabet=alphabet_var)
    with out_tree:
        clear_output()
        print('Prob dist: \n',widgetInst.P)
        print('Alphabet: \n', widgetInst.alphabet)
        print(widgetInst.create_tree())
        widgetInst.create_encoder()

btn.on_click(btn_eventhandler)

Button(description='Create Tree', style=ButtonStyle())

Output()

In [331]:
# text box that can write into and return encoded string

enc_text_widget = widgets.Text()
print('Text to be coded:')
display(enc_text_widget)

enc_btn = widgets.Button(description='Encode Text')
display(enc_btn)

out_enc = widgets.Output()
display(out_enc)

def enc_btn_eventhandler(obj):
    with out_enc:
        clear_output()
        enc_string = widgetInst.encode_string(enc_text_widget.value)
        if type(enc_text_widget.value) is str:
            print('Encoding for {} is {}'.format(enc_text_widget.value, enc_string))
enc_btn.on_click(enc_btn_eventhandler)

Text to be coded:


Text(value='')

Button(description='Encode Text', style=ButtonStyle())

Output()